<a href="https://colab.research.google.com/github/junhansong/AI_begin/blob/master/Order_%EC%A4%91%EA%B5%AD%EC%A7%91Bot_test_kor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앤드류 응의 'ChatGPT 프롬프트 엔지니어링'

- [ref](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)

  
    - 나만의 맞춤형 챗봇 구축하기
    - 한글 프롬프트 테스트

# 0_Setup
#### Load the API key and relevant Python libaries.

In this course, we've provided some code that loads the OpenAI API key for you.

In [ ]:
from getpass import getpass

OPENAI_API_KEY = getpass("OpenAI API Key : ")

OpenAI API Key : ··········


In [ ]:
# !pip install openai
!pip install openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
# import openai

import openai
# from openai import OpenAI

import os
import time

openai.api_key  = OPENAI_API_KEY

# client = OpenAI(api_key=OPENAI_API_KEY)

# 챗봇 만들기

Promt를 기반 챗봇

-[ref](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/8/chatbot)

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
    # response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
    # response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

## OrderBot

프롬프트 기반 사용자의 주문을 받는 봇 만들기



In [ ]:
!pip install panel

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})

    order_model = "gpt-4-turbo"
    # order_model = "gpt-3.5-turbo-16k"
    response = get_completion_from_messages(context, model=order_model)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display


# context = [ {'role':'system', 'content':"""
# 당신은 중국 음식점의 주문을 수집하는 자동화된 서비스인 OrderBot입니다.
# 먼저 고객에게 인사한 다음 주문을 받습니다.
# 전체 주문을 수령하기 위해 기다립니다.
# 메뉴는 확인해야 합니다.

# 그 후에 식당에서 먹을지, 포장인지, 배달인지 물어봅니다.
# 그런 다음 요약하고 고객이 추가로 원하는 것이 있는지 마지막으로 확인합니다.
# 식당에서 먹거나, 포장이면 시간을 물어보고, 배달이라면 주소를 물어보세요.
# 마지막으로 지불 방법을 확인합니다.
# 당신은 짧고 매우 대화하기 쉬운 스타일로 응답합니다.

# 식사와 요리 메뉴는 大, 中, 小 순서입니다.

# 메뉴에는 다음이 포함됩니다.

# 식사 :
#   짜장면  8000, 7000, 5000
#   짬뽕    10000, 8000, 6000

# 요리 :
#   탕수육 20000, 15000, 10000
#   양장피 30000, 20000, 15000
#   유산슬 40000, 30000, 20000

# 음료수:
#   콜라     2000
#   사이다   2000
#   소주     4000
#   막걸리   4000
#   맥주     5000

# """} ]  # accumulate messages


# 메뉴에 없는 것은 다른 메뉴 추천해.

context = [ {'role':'system', 'content':"""
당신은 중국 음식점의 주문을 수집하는 자동화된 서비스인 OrderBot입니다.
먼저 고객에게 인사한 다음 주문을 받습니다.
전체 주문을 수령하기 위해 기다립니다.
주문은 메뉴, size 를 확인해야 합니다.

그 후에 식당에서 먹을지, 포장인지, 배달인지 물어봅니다.
그런 다음 요약하고 고객이 추가로 원하는 것이 있는지 최종 확인합니다.

끝으로 식당에서 먹거나, 포장이면 시간을 물어보고, 배달이라면 주소를 물어보세요.
당신은 짧고 매우 대화하기 쉬운 스타일로 응답합니다.

식사와 요리 메뉴는 大, 中, 小 순서입니다.
없는 메뉴를 주문하면 다른 메뉴를 추천해줘.
고객이 추천을 요청하면, 날씨에 따른 추천 메뉴는 확인해서 추천해.



메뉴에는 다음이 포함됩니다.


식사 :
  짜장면  9000, 7000, 5000
  짜장밥  9000, 8000, 6000
  짬뽕    10000, 8000, 6000


요리 :
  탕수육 20000, 15000, 10000
  양장피 30000, 20000, 15000
  유산슬 40000, 30000, 20000
  두부김치  30000, 20000, 10000  (비올때 추천 메뉴)

음료수:
  콜라     2000
  사이다   2000
  소주     4000
  막걸리   4000
  맥주     5000

"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

<ipython-input-6-7a7e54d87032>:13: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

<ipython-input-6-7a7e54d87032>:13: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
<ipython-input-6-7a7e54d87032>:13: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
<ipython-input-6-7a7e54d87032>:13: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
<ipython-input-6-7a7e54d87032>:13: PanelDeprecationWarning: 'style' is deprecated and will be removed in version 1.4, use 'styles' instead.
  pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))


In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':
   """  차근차근 다음 결과를 주방에 전달합니다.
        한국어로 답해주세요.

        주문 내용을 다음 형식으로 표시합니다.
        1) 식사, 크기, 가격   목록
        2) 요리, 크기, 가격   목록
        3) 음료, 가격 목록
        4) 총 가격.
        5) 주소
        6) 시간
    """
 },
)


        # 5) 배달이면  주소 : 여기에 주소
        #    식당 식사, 포장 이면 시간 : 여기에 시간

 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

# response = get_completion_from_messages(messages, model="gpt-3.5-turbo-16k", temperature=0)
response = get_completion_from_messages(messages, model="gpt-4-1106-preview", temperature=0)
print(response)

주문 내용:

1) 식사: 짬뽕, 大, 10000원
2) 요리: 탕수육, 小, 10000원
3) 음료: 소주, 4000원
4) 총 가격: 24000원
5) 주소: 포장 주문
6) 시간: 오후 5시

주방에 전달 완료했습니다!


# Done!!!